In [1]:
import pandas as pd
import numpy as np

In [2]:
kf = pd.read_excel('C:/Users/ScMa9004/Documents/RI/RIEXPORT_31.01 (1).xlsx')

In [8]:
sf = pd.read_excel('C:/Users/ScMa9004/Documents/Справочник OPS_регион_QCT_2.2020.xlsx', sheet_name = 'данные')

In [4]:
def address_lists(kf, sf):
    
    kf = kf[kf['Stage'] == 'SELECTION'].reset_index(drop = True)
    
    kf['FMCG'] = np.nan
    kf['CIG'] = np.nan
    kf['BEER'] = np.nan
    kf['EXXON'] = np.nan
    kf['BST'] = np.nan
    kf['hip'] = np.nan
    kf['Cluster'] = np.nan
    kf['QCT'] = np.nan
    
    kf['FMCG'] = kf['BatchID'][kf['Index'] == 1000296]
    kf['CIG'] = kf['BatchID'][kf['Index'] == 1000334]
    kf['BEER'] = kf['BatchID'][kf['Index'] == 1000328]
    kf['EXXON'] = kf['BatchID'][kf['Index'] == 1001395]
    
    kf.loc[kf['FMCG'].notna(), 'BST'] = 1
    kf.loc[kf['CIG'].notna(), 'BST'] = 3
    kf.loc[kf['BEER'].notna(), 'BST'] = 4
    kf.loc[kf['EXXON'].notna(), 'BST'] = 5
    #kf.loc[kf['STORETYPE'] == 'EOS', 'BST'] = 6
    
    dup = pd.DataFrame()
    dup = kf[kf['GSRID'].duplicated(keep = False)]
    wf = pd.DataFrame()
    kf = kf[kf['GSRID'].isin(kf['GSRID'].drop_duplicates(keep = False))]
    
    for i in dup['GSRID'].unique():
        wf = dup[dup['GSRID'] == i]
        wf = wf.sort_values(by=['BST']).reset_index(drop = True)
        if 5 in wf['BST'].values:
            wf = wf.fillna(method='ffill')
            wf.loc[1, 'BST'] = 502
            kf = kf.append(wf.loc[1], ignore_index=True)
        else:
            wf = wf.fillna(method='bfill')
            if 1 not in wf['BST'].values:
                wf.loc[0, 'BST'] = 501
            kf = kf.append(wf.loc[0], ignore_index=True)
    
    for i in kf.index:
        kf.loc[i, 'Cluster'] = sf['ops'][sf['ccode'] == kf['CityCode'][i]].unique().item()
        kf.loc[i, 'QCT'] = sf['qct_name_NEW'][sf['ccode'] == kf['CityCode'][i]].unique().item()
        kf.loc[i, 'PROVINCE'] = sf['region'][sf['ccode'] == kf['CityCode'][i]].unique().item()   

####это удалить когда будет площадь и авторизованность добавлена в DFFU
        if 'Large Impulse stores' in kf['Cell Name'][i]:
            kf.loc[i, 'hip'] = 'IML'
        if 'Small Impulse stores' in kf['Cell Name'][i]:
            kf.loc[i, 'hip'] = 'IMS'
        if 'CDLself' in kf['Cell Name'][i] or 'SRVself' in kf['Cell Name'][i]:
            kf.loc[i, 'hip'] = 1
    
    kf.loc[kf['hip'].isna() & kf['DA_STORE_TYPE'].isin(['CDL', 'SRV', 'SPS', 'OPS']), 'hip'] = 0
####        
    kf = kf[['GSRID', 'SMPSEQ', 'FMCG', 'BEER', 'CIG', 'EXXON', 'Store Name', 'STORE_ADDRESS_DA', 'DA_STORE_TYPE', 'BST', 'Trade Type',\
             'hip', 'City/Regency', 'CityCode', 'Cluster', 'QCT', 'PROVINCE']]
                
    return kf

In [5]:
kf = address_lists(kf, sf)

In [6]:
df = pd.DataFrame()
df = kf[kf['SMPSEQ'] == 1].reset_index(drop = True)

In [7]:
with pd.ExcelWriter('Адресные списки.xlsx') as writer:
    df.to_excel(writer, sheet_name='основные адреса', index = False)
    kf.to_excel(writer, sheet_name='альтернативные адреса', index = False)